In [ ]:
import datetime
import json
import pandas as pd
import psycopg2
import os
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcpy import env

### Get raw data from SafeGraph database and create dataframes for the parks of interest
##### Victor Ashe Park & Lakeshore Park

In [ ]:
# Set up dataframe columns needed for analysis

fields = ['placekey', 'parent_placekey', 'location_name', 'street_address', 'city', 'region',
          'postal_code', 'safegraph_brand_ids', 'brands', 'date_range_start', 'date_range_end',
          'raw_visit_counts', 'raw_visitor_counts', 'visits_by_day', 'poi_cbg',
          'visitor_home_cbgs', 'visitor_daytime_cbgs', 'visitor_country_of_origin',
          'distance_from_home', 'median_dwell', 'bucketed_dwell_times', 'related_same_day_brand',
          'device_type']

field_str = ', '.join(fields)

In [ ]:
# Function to get dataframe with connection parameters

def get_dataframe(query):
    try:
        connection = psycopg2.connect(user='___________',
                                      password='___________',
                                      host='___________',
                                      port=___________,
                                      database='___________')
        connection.set_client_encoding("UTF8")
        print(connection.get_dsn_parameters(), "\n")
        cursor = connection.cursor()
        cursor.execute("SELECT version();")
        record = cursor.fetchone()
        print("You are connected to - ", record, "\n")
        with connection:
            cursor = connection.cursor()
            cursor.execute(query)
            rows = cursor.fetchall()
        data_frame = pd.DataFrame(rows, columns = fields)
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
    return data_frame

In [ ]:
lakeshore_park = get_dataframe(f"SELECT {field_str} FROM safegraph.weeklypatterns WHERE poi_cbg LIKE '47093%' AND location_name LIKE 'Lakeshore Park'")
lakeshore_park.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\lakeshore_park.csv")

lakeshore_park.head(5)

In [ ]:
victor_ashe_park = get_dataframe(f"SELECT {field_str} FROM safegraph.weeklypatterns WHERE poi_cbg LIKE '47093%' AND location_name LIKE 'Victor Ashe Park'")
victor_ashe_park.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\victor_ashe_park.csv")

victor_ashe_park.head()

### Read in processed datasets that will be used for the analyses

In [ ]:
# the _parse datasets are for the line graphs so that the dates show as a monthly value for the x axis
lakeshore_park_parse = pd.read_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\lakeshore_park_2019_2020.csv", parse_dates=['start_date'], dayfirst=False)
victor_ashe_parse = pd.read_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\victor_ashe_park_2019_2020.csv", parse_dates=['start_date'], dayfirst=False)

# these datasets are not parsed by the date and will be used for the bar charts
lakeshore_park = pd.read_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\lakeshore_park_2019_2020.csv")
victor_ashe = pd.read_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\victor_ashe_park_2019_2020.csv")

### Bar charts of the visit time ranges to each park
##### Lakeshore Park Dwell Time Charts

In [ ]:
fig = px.bar(lakeshore_park, x="start_date", y=['bucketed_dwell_times_lessthan5',
           'bucketed_dwell_times_5to10', 'bucketed_dwell_times_11to20',
           'bucketed_dwell_times_21to60', 'bucketed_dwell_times_61to120',
           'bucketed_dwell_times_121to240', 'bucketed_dwell_times_greaterthan240'],
           title="Length of Time in Minutes that Visitors Spent at Lakeshore Park")

newnames = {"bucketed_dwell_times_lessthan5": "less than 5 minutes",
           "bucketed_dwell_times_5to10": "5 to 10 minutes", 
           "bucketed_dwell_times_11to20": "11 to 20 minutes",
           "bucketed_dwell_times_21to60": "21 to 60 minutes", 
           "bucketed_dwell_times_61to120": "61 to 120 minutes",
           "bucketed_dwell_times_121to240": "121 to 240 minutes", 
           "bucketed_dwell_times_greaterthan240": "greater than 240 minutes"}


fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))

fig.update_layout(barmode='stack', legend_title="")
fig.update_yaxes(title_text='Visitor Count')
fig.update_xaxes(title_text='Week Starting Date')

# fig.update_layout(xaxis = go.layout.XAxis(tickangle = 45))

fig.show()

pio.write_image(fig, r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\images\Lakeshore_bucketed_visit_times_BarChart.png")

In [ ]:
fig = px.histogram(lakeshore_park, x="start_date", y=[' less than 5 minutes', ' 5 to 10 minutes ', ' 11 to 20 minutes',
       ' 21 to 60 minutes ', ' 61 to 120 minutes', ' 121 to 240 minutes ', ' greater than 240 minutes'], 
        title="Normalized Length of Time in Minutes that Visitors Spent at Lakeshore Park")

fig.update_layout(barmode='relative', legend_title="")

fig.update_yaxes(title_text='Percent Visitor Count', tickformat=".0%")
fig.update_xaxes(title_text='Week Starting Date')

# fig.update_layout(xaxis = go.layout.XAxis(tickangle = 45))

fig.show()

pio.write_image(fig, r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\images\Lakeshore_bucketed_visit_times_NormalizedBarChart.png")

##### Victor Ashe Park Dwell Time Charts

In [ ]:
fig = px.bar(victor_ashe, x="start_date", y=['bucketed_dwell_times_lessthan5',
           'bucketed_dwell_times_5to10', 'bucketed_dwell_times_11to20',
           'bucketed_dwell_times_21to60', 'bucketed_dwell_times_61to120',
           'bucketed_dwell_times_121to240', 'bucketed_dwell_times_greaterthan240'],
           title="Length of Time in Minutes that Visitors Spent at Victor Ashe Park")

newnames = {"bucketed_dwell_times_lessthan5": "less than 5 minutes",
           "bucketed_dwell_times_5to10": "5 to 10 minutes", 
           "bucketed_dwell_times_11to20": "11 to 20 minutes",
           "bucketed_dwell_times_21to60": "21 to 60 minutes", 
           "bucketed_dwell_times_61to120": "61 to 120 minutes",
           "bucketed_dwell_times_121to240": "121 to 240 minutes", 
           "bucketed_dwell_times_greaterthan240": "greater than 240 minutes"}


fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))

fig.update_layout(barmode='stack', legend_title="")
fig.update_yaxes(title_text='Visitor Count')
fig.update_xaxes(title_text='Week Starting Date')

# fig.update_layout(xaxis = go.layout.XAxis(tickangle = 45))

fig.show()

pio.write_image(fig, r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\images\Victor_Ashe_bucketed_visit_times_BarChart.png")

In [ ]:
fig = px.histogram(victor_ashe, x="start_date", y=[' less than 5 minutes', ' 5 to 10 minutes ', ' 11 to 20 minutes',
       ' 21 to 60 minutes ', ' 61 to 120 minutes', ' 121 to 240 minutes ', ' greater than 240 minutes'], 
        title="Normalized Length of Time in Minutes that Visitors Spent at Victor Ashe Park")

fig.update_layout(barmode='relative', legend_title="")

fig.update_yaxes(title_text='Percent Visitor Count', tickformat=".0%")
fig.update_xaxes(title_text='Week Starting Date')

# fig.update_layout(xaxis = go.layout.XAxis(tickangle = 45))

fig.show()

pio.write_image(fig, r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\images\Victor_Ashe_bucketed_visit_times_NormalizedBarChart.png")

### Create the seasonal datasets from the full dataset for each park
##### Lakeshore Park Seasonal Dataset Processing and Statistics

In [ ]:
lakeshore_winter_2018 = lakeshore_park[lakeshore_park['season'] == 'Winter']
lakeshore_spring_2019 = lakeshore_park[lakeshore_park['season'] == 'Spring']
lakeshore_summer_2019 = lakeshore_park[lakeshore_park['season'] == 'Summer']
lakeshore_fall_2019 = lakeshore_park[lakeshore_park['season'] == 'Fall']

lakeshore_winter_2019 = lakeshore_park[lakeshore_park['season'] == 'Winter 2']
lakeshore_spring_2020 = lakeshore_park[lakeshore_park['season'] == 'Spring 2']
lakeshore_summer_2020 = lakeshore_park[lakeshore_park['season'] == 'Summer 2']
lakeshore_fall_2020 = lakeshore_park[lakeshore_park['season'] == 'Fall 2']

In [ ]:
lakeshore_winter_2018.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_winter_2018.csv")
lakeshore_winter_2018.raw_visit_counts.describe()

In [ ]:
lakeshore_spring_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_spring_2019.csv")
lakeshore_spring_2019.raw_visit_counts.describe()

In [ ]:
lakeshore_summer_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_summer_2019.csv")
lakeshore_summer_2019.raw_visit_counts.describe()

In [ ]:
lakeshore_fall_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_fall_2019.csv")
lakeshore_fall_2019.raw_visit_counts.describe()

In [ ]:
lakeshore_winter_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_winter_2019.csv")
lakeshore_winter_2019.raw_visit_counts.describe()

In [ ]:
lakeshore_spring_2020.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_spring_2020.csv")
lakeshore_spring_2020.raw_visit_counts.describe()

In [ ]:
lakeshore_summer_2020.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_summer_2020.csv")
lakeshore_summer_2020.raw_visit_counts.describe()

In [ ]:
lakeshore_fall_2020.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\lakeshore_fall_2020.csv")
lakeshore_fall_2020.raw_visit_counts.describe()

##### Victor Ashe Seasonal Dataset Processing and Statistics

In [ ]:
victor_ashe_winter_2018 = victor_ashe[victor_ashe['season'] == 'Winter']
victor_ashe_spring_2019 = victor_ashe[victor_ashe['season'] == 'Spring']
victor_ashe_summer_2019 = victor_ashe[victor_ashe['season'] == 'Summer']
victor_ashe_fall_2019 = victor_ashe[victor_ashe['season'] == 'Fall']

victor_ashe_winter_2019 = victor_ashe[victor_ashe['season'] == 'Winter 2']
victor_ashe_spring_2020 = victor_ashe[victor_ashe['season'] == 'Spring 2']
victor_ashe_summer_2020 = victor_ashe[victor_ashe['season'] == 'Summer 2']
victor_ashe_fall_2020 = victor_ashe[victor_ashe['season'] == 'Fall 2']

In [ ]:
victor_ashe_winter_2018.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_winter_2018.csv")
victor_ashe_winter_2018.raw_visit_counts.describe()

In [ ]:
victor_ashe_spring_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_spring_2019.csv")
victor_ashe_spring_2019.raw_visit_counts.describe()

In [ ]:
victor_ashe_summer_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_summer_2019.csv")
victor_ashe_summer_2019.raw_visit_counts.describe()

In [ ]:
victor_ashe_fall_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_fall_2019.csv")
victor_ashe_fall_2019.raw_visit_counts.describe()

In [ ]:
victor_ashe_winter_2019.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_winter_2019.csv")
victor_ashe_winter_2019.raw_visit_counts.describe()

In [ ]:
victor_ashe_spring_2020.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_spring_2020.csv")
victor_ashe_spring_2020.raw_visit_counts.describe()

In [ ]:
victor_ashe_summer_2020.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_summer_2020.csv")
victor_ashe_summer_2020.raw_visit_counts.describe()

In [ ]:
victor_ashe_fall_2020.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Data\SafeGraph Datasets\final_spreadsheets\by season\victor_ashe_fall_2020.csv")
victor_ashe_fall_2020.raw_visit_counts.describe()

### Graphs of the raw visit counts for each park

In [ ]:
fig = px.line(lakeshore_park_parse, x="start_date", 
              y="raw_visit_counts", 
              color="season",
              line_shape='spline',
              markers=True,
              color_discrete_map={
                  "Spring": "green",
                  "Spring 2": "green",
                  "Summer": "red",
                  "Summer 2": "red",
                  "Fall": "purple",
                  "Fall 2": "purple",
                  "Winter": "blue",
                  "Winter 2": "blue"})


fig.update_layout({"title": '<b>''Lakeshore Park SafeGraph Seasonal Visitation <br> Pre-COVID and During COVID''</b>',
                  "title_x": 0.5,
                  "xaxis": {"title":"Months"},
                  "yaxis": {"title":"Total Park Visits"},
                  "showlegend": True})

for trace in fig['data']:
    if(trace['name'] == 'Spring 2'): trace['showlegend'] = False
        
for trace in fig['data']:
    if(trace['name'] == 'Summer 2'): trace['showlegend'] = False
        
for trace in fig['data']:
    if(trace['name'] == 'Fall 2'): trace['showlegend'] = False
        
for trace in fig['data']:
    if(trace['name'] == 'Winter 2'): trace['showlegend'] = False

fig.update_layout(legend_title="Seasons")
fig.update_yaxes(range = [0, 1100], title_text = "Visitor Count") 
fig.update_xaxes(title_text = "")
fig.show()

pio.write_image(fig, r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\images\Lakeshore_Seasonal_SafeGraph_LineGraph.png")

In [ ]:
fig = px.line(victor_ashe_parse, x="start_date", 
              y="raw_visit_counts", 
              color="season",
              line_shape='spline',
              markers=True,
              color_discrete_map={
                  "Spring": "green",
                  "Spring 2": "green",
                  "Summer": "red",
                  "Summer 2": "red",
                  "Fall": "purple",
                  "Fall 2": "purple",
                  "Winter": "blue",
                  "Winter 2": "blue"})


fig.update_layout({"title": '<b>''Victor Ashe Park SafeGraph Seasonal Visitation Rates <br> Pre-COVID and During COVID''</b>',
                  "title_x": 0.5,
                  "xaxis": {"title":"Months"},
                  "yaxis": {"title":"Total Park Visits"},
                  "showlegend": True})

for trace in fig['data']:
    if(trace['name'] == 'Spring 2'): trace['showlegend'] = False
        
for trace in fig['data']:
    if(trace['name'] == 'Summer 2'): trace['showlegend'] = False
        
for trace in fig['data']:
    if(trace['name'] == 'Fall 2'): trace['showlegend'] = False
        
for trace in fig['data']:
    if(trace['name'] == 'Winter 2'): trace['showlegend'] = False
        
fig.update_layout(legend_title="Seasons")
fig.update_yaxes(range = [0, 3700], title_text = "Visitor Count") 
fig.update_xaxes(title_text = "")

fig.show()

pio.write_image(fig, r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\images\Victor_Ashe_Seasonal_SafeGraph_LineGraph.png")

### Get the cbgs count for each season in an excel file
##### Lakeshore Park

In [ ]:
lakeshore_winter_2018_dict  = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_winter_2018_dict[k] = lakeshore_winter_2018_dict.setdefault(k, 0) + v
        
lakeshore_winter_2018['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_winter_2018_dict

lakeshore_winter_2018_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_winter_2018_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_winter_2018_dict.values()]})

lakeshore_winter_2018_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_winter_2018_cbgs.xlsx")

In [ ]:
lakeshore_spring_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_spring_2019_dict[k] = lakeshore_spring_2019_dict.setdefault(k, 0) + v
        
lakeshore_spring_2019['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_spring_2019_dict

lakeshore_spring_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_spring_2019_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_spring_2019_dict.values()]})

lakeshore_spring_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_spring_2019_cbgs.xlsx")

In [ ]:
lakeshore_summer_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_summer_2019_dict[k] = lakeshore_summer_2019_dict.setdefault(k, 0) + v
        
lakeshore_summer_2019['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_summer_2019_dict

lakeshore_summer_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_summer_2019_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_summer_2019_dict.values()]})

lakeshore_summer_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_summer_2019_cbgs.xlsx")

In [ ]:
lakeshore_fall_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_fall_2019_dict[k] = lakeshore_fall_2019_dict.setdefault(k, 0) + v
        
lakeshore_fall_2019['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_fall_2019_dict

lakeshore_fall_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_fall_2019_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_fall_2019_dict.values()]})

lakeshore_fall_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_fall_2019_cbgs.xlsx")

In [ ]:
lakeshore_winter_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_winter_2019_dict[k] = lakeshore_winter_2019_dict.setdefault(k, 0) + v
        
lakeshore_winter_2019['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_winter_2019_dict

lakeshore_winter_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_winter_2019_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_winter_2019_dict.values()]})

lakeshore_winter_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_winter_2019_cbgs.xlsx")

In [ ]:
lakeshore_spring_2020_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_spring_2020_dict[k] = lakeshore_spring_2020_dict.setdefault(k, 0) + v
        
lakeshore_spring_2020['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_spring_2020_dict

lakeshore_spring_2020_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_spring_2020_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_spring_2020_dict.values()]})

lakeshore_spring_2020_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_spring_2020_cbgs.xlsx")

In [ ]:
lakeshore_summer_2020_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_summer_2020_dict[k] = lakeshore_summer_2020_dict.setdefault(k, 0) + v
        
lakeshore_summer_2020['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_summer_2020_dict

lakeshore_summer_2020_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_summer_2020_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_summer_2020_dict.values()]})

lakeshore_summer_2020_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_summer_2020_cbgs.xlsx")

In [ ]:
lakeshore_fall_2020_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        lakeshore_fall_2020_dict[k] = lakeshore_fall_2020_dict.setdefault(k, 0) + v
        
lakeshore_fall_2020['visitor_home_cbgs'].apply(iter_thru_logs)

lakeshore_fall_2020_dict

lakeshore_fall_2020_df_out = pd.DataFrame({'cbgs_id': [x for x in lakeshore_fall_2020_dict.keys()],
                       'cbgs_count': [y for y in lakeshore_fall_2020_dict.values()]})

lakeshore_fall_2020_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_fall_2020_cbgs.xlsx")

##### Victor Ashe Park

In [ ]:
victor_ashe_winter_2018_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_winter_2018_dict[k] = victor_ashe_winter_2018_dict.setdefault(k, 0) + v
        
victor_ashe_winter_2018['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_winter_2018_dict

victor_ashe_winter_2018_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_winter_2018_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_winter_2018_dict.values()]})

victor_ashe_winter_2018_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_winter_2018_cbgs.xlsx")

In [ ]:
victor_ashe_spring_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_spring_2019_dict[k] = victor_ashe_spring_2019_dict.setdefault(k, 0) + v
        
victor_ashe_spring_2019['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_spring_2019_dict

victor_ashe_spring_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_spring_2019_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_spring_2019_dict.values()]})

victor_ashe_spring_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_spring_2019_cbgs.xlsx")

In [ ]:
victor_ashe_summer_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_summer_2019_dict[k] = victor_ashe_summer_2019_dict.setdefault(k, 0) + v
        
victor_ashe_summer_2019['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_summer_2019_dict

victor_ashe_summer_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_summer_2019_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_summer_2019_dict.values()]})

victor_ashe_summer_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_summer_2019_cbgs.xlsx")

In [ ]:
victor_ashe_fall_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_fall_2019_dict[k] = victor_ashe_fall_2019_dict.setdefault(k, 0) + v
        
victor_ashe_fall_2019['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_fall_2019_dict

victor_ashe_fall_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_fall_2019_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_fall_2019_dict.values()]})

victor_ashe_fall_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_fall_2019_cbgs.xlsx")

In [ ]:
victor_ashe_winter_2019_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_winter_2019_dict[k] = victor_ashe_winter_2019_dict.setdefault(k, 0) + v
        
victor_ashe_winter_2019['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_winter_2019_dict

victor_ashe_winter_2019_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_winter_2019_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_winter_2019_dict.values()]})

victor_ashe_winter_2019_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_winter_2019_cbgs.xlsx")

In [ ]:
victor_ashe_spring_2020_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_spring_2020_dict[k] = victor_ashe_spring_2020_dict.setdefault(k, 0) + v
        
victor_ashe_spring_2020['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_spring_2020_dict

victor_ashe_spring_2020_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_spring_2020_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_spring_2020_dict.values()]})

victor_ashe_spring_2020_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_spring_2020_cbgs.xlsx")

In [ ]:
victor_ashe_summer_2020_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_summer_2020_dict[k] = victor_ashe_summer_2020_dict.setdefault(k, 0) + v
        
victor_ashe_summer_2020['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_summer_2020_dict

victor_ashe_summer_2020_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_summer_2020_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_summer_2020_dict.values()]})

victor_ashe_summer_2020_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_summer_2020_cbgs.xlsx")

In [ ]:
victor_ashe_fall_2020_dict = {}

def iter_thru_logs(log):
    log = json.loads(log.replace("'", '"'))
    for k, v in log.items():
        victor_ashe_fall_2020_dict[k] = victor_ashe_fall_2020_dict.setdefault(k, 0) + v
        
victor_ashe_fall_2020['visitor_home_cbgs'].apply(iter_thru_logs)

victor_ashe_fall_2020_dict

victor_ashe_fall_2020_df_out = pd.DataFrame({'cbgs_id': [x for x in victor_ashe_fall_2020_dict.keys()],
                       'cbgs_count': [y for y in victor_ashe_fall_2020_dict.values()]})

victor_ashe_fall_2020_df_out["cbgs_text"] = victor_ashe_fall_2020_df_out["cbgs_id"].astype(str)

victor_ashe_fall_2020_df_out.to_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_fall_2020_cbgs.xlsx")

### Knox COVID-19 Counts

In [ ]:
covid_counts = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\COVID19 data\Public-Dataset-County-New.xlsx")

names = ["Anderson", "Blount", "Grainger", "Jefferson", "Knox", "Loudon", "Roane", "Sevier", "Union"]
knox_counties = covid_counts[covid_counts["COUNTY"].isin(names)]

knox_covid_by_date = knox_counties.groupby('DATE').sum()
knox_covid_by_date.to_csv(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\COVID19 data\covid_knox.csv")

knox_covid_simple = pd.read_csv(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\COVID19 data\covid_knox_simple.csv", parse_dates=['DATE'], dayfirst=False)
knox_covid_simple.columns

In [1]:
# Create COVID-19 Graph
fig = px.line(knox_covid_simple, x="DATE", 
              y="NEW_CASES")


fig.update_layout({"title": '<b>New COVID-19 Cases for Knoxville, Tennessee',
                  "title_x": 0.5,
                  "xaxis": {"title":"Months"},
                  "yaxis": {"title":"New COVID-19 Cases"},
                  "showlegend": True})

fig.update_traces(line_color='#000000', line_width=1.5)

# fig.update_layout(legend_title="Seasons")
# fig.update_yaxes(range = [0, 1100], title_text = "Visitor Count") 
fig.update_xaxes(title_text = "")
fig.show()

io.write_image(fig, r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\images\Knoxville_NewCOVID19Cases1_5.png")

### Process all the cbgs count dataframes

In [ ]:
lakeshore_winter_2018_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_winter_2018_cbgs.xlsx")
lakeshore_spring_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_spring_2019_cbgs.xlsx")
lakeshore_summer_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_summer_2019_cbgs.xlsx")
lakeshore_fall_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_fall_2019_cbgs.xlsx")
lakeshore_winter_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_winter_2019_cbgs.xlsx")
lakeshore_spring_2020_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_spring_2020_cbgs.xlsx")
lakeshore_summer_2020_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_summer_2020_cbgs.xlsx")
lakeshore_fall_2020_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\lakeshore_fall_2020_cbgs.xlsx")

victor_ashe_winter_2018_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_winter_2018_cbgs.xlsx")
victor_ashe_spring_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_spring_2019_cbgs.xlsx")
victor_ashe_summer_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_summer_2019_cbgs.xlsx")
victor_ashe_fall_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_fall_2019_cbgs.xlsx")
victor_ashe_winter_2019_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_winter_2019_cbgs.xlsx")
victor_ashe_spring_2020_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_spring_2020_cbgs.xlsx")
victor_ashe_summer_2020_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_summer_2020_cbgs.xlsx")
victor_ashe_fall_2020_df_out = pd.read_excel(r"C:\Users\18f\Desktop\Capstone Project\Jupyter Notebook\Capstone_Project_Data\datasets\cbgs_counts_by_season\victor_ashe_fall_2020_cbgs.xlsx")

In [ ]:
# directory = r"Y:\18f_Chantelle_Rittmaier\capstone project"

# arcpy.CreateFileGDB_management(r"Y:\18f_Chantelle_Rittmaier\capstone project", "capstone_gdb.gdb")

In [ ]:
env.workspace = r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb"

cbgs_projected = r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\cbgs_projected.shp" 

fcList = arcpy.ListFeatureClasses()

for fc in fcList:
    print(fc)

In [ ]:
tables = arcpy.ListTables()

for table in tables:
    print(table)

In [ ]:
# copy the cbgs polygon shapefile
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2019_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2020_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2019_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2020_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2019_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2020_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2018_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2019_cbgs_copy")

arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2019_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2020_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2019_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2020_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2019_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2020_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2018_cbgs_copy")
arcpy.management.CopyFeatures("cbgs_projected", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2019_cbgs_copy")


In [ ]:
# copy the cbgs point layer for each season
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2019_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2020_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2019_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2020_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2019_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2020_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2018_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2019_cbgspoints_copy")

arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2019_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2020_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2019_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2020_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2019_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2020_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2018_cbgspoints_copy")
arcpy.management.CopyFeatures("cbgs_points", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2019_cbgspoints_copy")

In [ ]:
# join table to the point layer based on the GEOID
join_table = arcpy.management.AddJoin("ls_fall_2019_cbgspoints_copy", "GEOID", "lakeshore_fall_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2019_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("ls_fall_2020_cbgspoints_copy", "GEOID", "lakeshore_fall_2020", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2020_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("ls_spring_2019_cbgspoints_copy", "GEOID", "lakeshore_spring_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2019_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("ls_spring_2020_cbgspoints_copy", "GEOID", "lakeshore_spring_2020", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2020_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("ls_summer_2019_cbgspoints_copy", "GEOID", "lakeshore_summer_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2019_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("ls_summer_2020_cbgspoints_copy", "GEOID", "lakeshore_summer_2020", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2020_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("ls_winter_2018_cbgspoints_copy", "GEOID", "lakeshore_winter_2018", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2018_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("ls_winter_2019_cbgspoints_copy", "GEOID", "lakeshore_winter_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2019_cbgspoints_joincopy")



join_table = arcpy.management.AddJoin("va_fall_2019_cbgspoints_copy", "GEOID", "victor_ashe_fall_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2019_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("va_fall_2020_cbgspoints_copy", "GEOID", "victor_ashe_fall_2020", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2020_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("va_spring_2019_cbgspoints_copy", "GEOID", "victor_ashe_spring_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2019_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("va_spring_2020_cbgspoints_copy", "GEOID", "victor_ashe_spring_2020", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2020_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("va_summer_2019_cbgspoints_copy", "GEOID", "victor_ashe_summer_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2019_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("va_summer_2020_cbgspoints_copy", "GEOID", "victor_ashe_summer_2020", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2020_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("va_winter_2018_cbgspoints_copy", "GEOID", "victor_ashe_winter_2018", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2018_cbgspoints_joincopy")

join_table = arcpy.management.AddJoin("va_winter_2019_cbgspoints_copy", "GEOID", "victor_ashe_winter_2019", "cbgs_id")
arcpy.management.CopyFeatures(join_table, r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2019_cbgspoints_joincopy")

In [ ]:
# create a new field called "norm_count"
arcpy.AddField_management("ls_fall_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("ls_fall_2020_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("ls_spring_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("ls_spring_2020_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("ls_summer_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("ls_summer_2020_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("ls_winter_2018_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("ls_winter_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")

arcpy.AddField_management("va_fall_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("va_fall_2020_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("va_spring_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("va_spring_2020_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("va_summer_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("va_summer_2020_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("va_winter_2018_cbgspoints_joincopy", "norm_ct", "FLOAT")
arcpy.AddField_management("va_winter_2019_cbgspoints_joincopy", "norm_ct", "FLOAT")

In [ ]:
with arcpy.da.UpdateCursor("ls_fall_2019_cbgspoints_joincopy", "lakeshore_fall_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_fall_2020_cbgspoints_joincopy", "lakeshore_fall_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("ls_spring_2019_cbgspoints_joincopy", "lakeshore_spring_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("ls_spring_2020_cbgspoints_joincopy", "lakeshore_spring_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_summer_2019_cbgspoints_joincopy", "lakeshore_summer_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_summer_2020_cbgspoints_joincopy", "lakeshore_summer_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_winter_2018_cbgspoints_joincopy", "lakeshore_winter_2018_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
        
with arcpy.da.UpdateCursor("ls_winter_2019_cbgspoints_joincopy", "lakeshore_winter_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
            
            
            
            
            
with arcpy.da.UpdateCursor("va_fall_2019_cbgspoints_joincopy", "victor_ashe_fall_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_fall_2020_cbgspoints_joincopy", "victor_ashe_fall_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("va_spring_2019_cbgspoints_joincopy", "victor_ashe_spring_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("va_spring_2020_cbgspoints_joincopy", "victor_ashe_spring_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_summer_2019_cbgspoints_joincopy", "victor_ashe_summer_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_summer_2020_cbgspoints_joincopy", "victor_ashe_summer_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_winter_2018_cbgspoints_joincopy", "victor_ashe_winter_2018_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
        
with arcpy.da.UpdateCursor("va_winter_2019_cbgspoints_joincopy", "victor_ashe_winter_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

In [ ]:
# use the field calculator to calculate the normalized count by population (P0010001)
arcpy.CalculateField_management("ls_fall_2019_cbgspoints_joincopy", "norm_ct", '!lakeshore_fall_2019_cbgs_count! / !ls_fall_2019_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("ls_fall_2020_cbgspoints_joincopy", "norm_ct", '!lakeshore_fall_2020_cbgs_count! / !ls_fall_2020_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("ls_spring_2019_cbgspoints_joincopy", "norm_ct", '!lakeshore_spring_2019_cbgs_count! / !ls_spring_2019_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("ls_spring_2020_cbgspoints_joincopy", "norm_ct", '!lakeshore_spring_2020_cbgs_count! / !ls_spring_2020_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("ls_summer_2019_cbgspoints_joincopy", "norm_ct", '!lakeshore_summer_2019_cbgs_count! / !ls_summer_2019_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("ls_summer_2020_cbgspoints_joincopy", "norm_ct", '!lakeshore_summer_2020_cbgs_count! / !ls_summer_2020_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("ls_winter_2018_cbgspoints_joincopy", "norm_ct", '!lakeshore_winter_2018_cbgs_count! / !ls_winter_2018_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("ls_winter_2019_cbgspoints_joincopy", "norm_ct", '!lakeshore_winter_2019_cbgs_count! / !ls_winter_2019_cbgspoints_copy_P0010001!', "PYTHON")

arcpy.CalculateField_management("va_fall_2019_cbgspoints_joincopy", "norm_ct", '!victor_ashe_fall_2019_cbgs_count! / !va_fall_2019_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("va_fall_2020_cbgspoints_joincopy", "norm_ct", '!victor_ashe_fall_2020_cbgs_count! / !va_fall_2020_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("va_spring_2019_cbgspoints_joincopy", "norm_ct", '!victor_ashe_spring_2019_cbgs_count! / !va_spring_2019_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("va_spring_2020_cbgspoints_joincopy", "norm_ct", '!victor_ashe_spring_2020_cbgs_count! / !va_spring_2020_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("va_summer_2019_cbgspoints_joincopy", "norm_ct", '!victor_ashe_summer_2019_cbgs_count! / !va_summer_2019_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("va_summer_2020_cbgspoints_joincopy", "norm_ct", '!victor_ashe_summer_2020_cbgs_count! / !va_summer_2020_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("va_winter_2018_cbgspoints_joincopy", "norm_ct", '!victor_ashe_winter_2018_cbgs_count! / !va_winter_2018_cbgspoints_copy_P0010001!', "PYTHON")
arcpy.CalculateField_management("va_winter_2019_cbgspoints_joincopy", "norm_ct", '!victor_ashe_winter_2019_cbgs_count! / !va_winter_2019_cbgspoints_copy_P0010001!', "PYTHON")

In [ ]:
# calculate the "near distance" from the point layer to the park point in meters
arcpy.analysis.Near("ls_fall_2019_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("ls_fall_2020_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("ls_spring_2019_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("ls_spring_2020_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("ls_summer_2019_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("ls_summer_2020_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("ls_winter_2018_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("ls_winter_2019_cbgspoints_joincopy", "lakeshore_park_centroid")

arcpy.analysis.Near("va_fall_2019_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("va_fall_2020_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("va_spring_2019_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("va_spring_2020_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("va_summer_2019_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("va_summer_2020_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("va_winter_2018_cbgspoints_joincopy", "lakeshore_park_centroid")
arcpy.analysis.Near("va_winter_2019_cbgspoints_joincopy", "lakeshore_park_centroid")

In [ ]:
# join the point layer to the polygon shapefile based on location
arcpy.SpatialJoin_analysis("ls_fall_2019_cbgs_copy", "ls_fall_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2019_spatialjoin")
arcpy.SpatialJoin_analysis("ls_fall_2020_cbgs_copy", "ls_fall_2020_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_fall_2020_spatialjoin")
arcpy.SpatialJoin_analysis("ls_spring_2019_cbgs_copy", "ls_spring_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2019_spatialjoin")
arcpy.SpatialJoin_analysis("ls_spring_2020_cbgs_copy", "ls_spring_2020_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_spring_2020_spatialjoin")
arcpy.SpatialJoin_analysis("ls_summer_2019_cbgs_copy", "ls_summer_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2019_spatialjoin")
arcpy.SpatialJoin_analysis("ls_summer_2020_cbgs_copy", "ls_summer_2020_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_summer_2020_spatialjoin")
arcpy.SpatialJoin_analysis("ls_winter_2018_cbgs_copy", "ls_winter_2018_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2018_spatialjoin")
arcpy.SpatialJoin_analysis("ls_winter_2019_cbgs_copy", "ls_winter_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\ls_winter_2019_spatialjoin")

arcpy.SpatialJoin_analysis("va_fall_2019_cbgs_copy", "va_fall_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2019_spatialjoin")
arcpy.SpatialJoin_analysis("va_fall_2020_cbgs_copy", "va_fall_2020_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_fall_2020_spatialjoin")
arcpy.SpatialJoin_analysis("va_spring_2019_cbgs_copy", "va_spring_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2019_spatialjoin")
arcpy.SpatialJoin_analysis("va_spring_2020_cbgs_copy", "va_spring_2020_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_spring_2020_spatialjoin")
arcpy.SpatialJoin_analysis("va_summer_2019_cbgs_copy", "va_summer_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2019_spatialjoin")
arcpy.SpatialJoin_analysis("va_summer_2020_cbgs_copy", "va_summer_2020_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_summer_2020_spatialjoin")
arcpy.SpatialJoin_analysis("va_winter_2018_cbgs_copy", "va_winter_2018_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2018_spatialjoin")
arcpy.SpatialJoin_analysis("va_winter_2019_cbgs_copy", "va_winter_2019_cbgspoints_joincopy", r"Y:\18f_Chantelle_Rittmaier\capstone project\capstone_gdb.gdb\va_winter_2019_spatialjoin")

In [ ]:
# delete any of the census block group tracts that do not have a cbgs count for the number of visitors
with arcpy.da.UpdateCursor("ls_fall_2019_spatialjoin", "lakeshore_fall_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_fall_2020_spatialjoin", "lakeshore_fall_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("ls_spring_2019_spatialjoin", "lakeshore_spring_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("ls_spring_2020_spatialjoin", "lakeshore_spring_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_summer_2019_spatialjoin", "lakeshore_summer_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_summer_2020_spatialjoin", "lakeshore_summer_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("ls_winter_2018_spatialjoin", "lakeshore_winter_2018_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
        
with arcpy.da.UpdateCursor("ls_winter_2019_spatialjoin", "lakeshore_winter_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
            
            
            
            
            
with arcpy.da.UpdateCursor("va_fall_2019_spatialjoin", "victor_ashe_fall_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_fall_2020_spatialjoin", "victor_ashe_fall_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("va_spring_2019_spatialjoin", "victor_ashe_spring_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

with arcpy.da.UpdateCursor("va_spring_2020_spatialjoin", "victor_ashe_spring_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_summer_2019_spatialjoin", "victor_ashe_summer_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_summer_2020_spatialjoin", "victor_ashe_summer_2020_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
            
with arcpy.da.UpdateCursor("va_winter_2018_spatialjoin", "victor_ashe_winter_2018_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()
        
with arcpy.da.UpdateCursor("va_winter_2019_spatialjoin", "victor_ashe_winter_2019_cbgs_count") as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

### Create graphs of the OLS Regressions
##### The OLS analyses were completed in ArcPro

In [ ]:
Lakeshore_Fall_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Fall_2019_OLS_Results_TableToExcel.xls")
Lakeshore_Fall_2020_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Fall_2020_OLS_Results_TableToExcel.xls")
Lakeshore_Spring_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Spring_2019_OLS_Results_TableToExcel.xls")
Lakeshore_Spring_2020_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Spring_2020_OLS_Results_TableToExcel.xls")
Lakeshore_Summer_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Summer_2019_OLS_Results_TableToExcel.xls")
Lakeshore_Summer_2020_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Summer_2020_OLS_Results_TableToExcel.xls")
Lakeshore_Winter_2018_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Winter_2018_OLS_Results_TableToExcel.xls")
Lakeshore_Winter_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Lakeshore_Winter_2019_OLS_Results_TableToExcel.xls")

Victor_Ashe_Fall_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Fall_2019_OLS_Results_TableToExcel.xls")
Victor_Ashe_Fall_2020_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Fall_2020_OLS_Results_TableToExcel.xls")
Victor_Ashe_Spring_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Spring_2019_OLS_Results_TableToExcel.xls")
Victor_Ashe_Spring_2020_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Spring_2020_OLS_Results_TableToExcel.xls")
Victor_Ashe_Summer_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Summer_2019_OLS_Results_TableToExcel.xls")
Victor_Ashe_Summer_2020_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Summer_2020_OLS_Results_TableToExcel.xls")
Victor_Ashe_Winter_2018_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Winter_2018_OLS_Results_TableToExcel.xls")
Victor_Ashe_Winter_2019_OLS = pd.read_excel(r"Y:\18f_Chantelle_Rittmaier\capstone project\OLS_Figures_Maps\OLS Tables\Victor_Ashe_Winter_2019_OLS_Results_TableToExcel.xls")

##### Lakeshore Park

In [ ]:
fig = px.scatter(Lakeshore_Fall_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Lakeshore_Fall_2020_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Lakeshore_Spring_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Lakeshore_Spring_2020_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Lakeshore_Summer_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Lakeshore_Summer_2020_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Lakeshore_Winter_2018_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Lakeshore_Winter_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

##### Victor Ashe Park

In [ ]:
fig = px.scatter(Victor_Ashe_Fall_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Victor_Ashe_Fall_2020_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Victor_Ashe_Spring_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Victor_Ashe_Spring_2020_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Victor_Ashe_Summer_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Victor_Ashe_Summer_2020_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Victor_Ashe_Winter_2018_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

In [ ]:
fig = px.scatter(Victor_Ashe_Winter_2019_OLS, x="NEAR_DIST_mile", y="norm_ct", trendline="ols", trendline_options=dict(log_x=True))

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
print(results)